In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

from imblearn.over_sampling import RandomOverSampler


Bad key "text.kerning_factor" on line 4 in
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint

In [2]:
all_flights = pd.read_pickle('/Users/wasilaq/Metis/Proj3/pickled_files/all_flights')

In [3]:
all_flights.head()

,dayofmonth_x,dayofweek,flightdate,marketing_airline_network,flight_number_marketing_airline,operating_airline,tail_number,flight_number_operating_airline,origin,origincityname,...,dep,dayofmonth_y,eventid,Type,severity,StartTime(UTC),EndTime(UTC),dayofmonth_y,start_time,end_time
0,29,1,2018-01-29,UA,421,UA,N489UA,421,ATL,"Atlanta, GA",...,2018-01-29 10:10:00,NaN,NaN,None,None,NaT,NaT,NaN,NaN,NaN
1,29,1,2018-01-29,UA,356,UA,N843UA,356,ATL,"Atlanta, GA",...,2018-01-29 13:13:00,NaN,NaN,None,None,NaT,NaT,NaN,NaN,NaN
2,29,1,2018-01-29,UA,313,UA,N453UA,313,ATL,"Atlanta, GA",...,2018-01-29 19:19:00,NaN,NaN,None,None,NaT,NaT,NaN,NaN,NaN
3,29,1,2018-01-29,UA,311,UA,N816UA,311,ATL,"Atlanta, GA",...,2018-01-29 07:07:00,29,W-3204100,Rain,Light,2018-01-29 03:52:00,2018-01-29 08:52:00,29,1.51722e+09,1.51722e+09
4,29,1,2018-01-29,UA,258,UA,N890UA,258,ATL,"Atlanta, GA",...,2018-01-29 17:17:00,NaN,NaN,None,None,NaT,NaT,NaN,NaN,NaN


In [8]:
all_flights.columns

Index(['dayofmonth_x', 'dayofweek', 'flightdate', 'marketing_airline_network',
       'flight_number_marketing_airline', 'operating_airline', 'tail_number',
       'flight_number_operating_airline', 'origin', 'origincityname',
       'originstate', 'dest', 'destcityname', 'deststate', 'deptime',
       'depdel15', 'deptimeblk', 'taxiout', 'taxiin', 'arrtime', 'arrdel15',
       'arrtimeblk', 'airtime', 'distance', 'arrtime2', 'deptime2', 'dep',
       'dayofmonth_y', 'eventid', 'Type', 'severity', 'StartTime(UTC)',
       'EndTime(UTC)', 'dayofmonth_y', 'start_time', 'end_time'],
      dtype='object')

In [14]:
# use one destination feature
dest_features = ['dest', 'destcityname', 'deststate']

time_features = ['airtime','taxiout','taxiin','depdel15']

features = [
    'dayofweek','marketing_airline_network','operating_airline','deptimeblk','arrtimeblk',
    'distance','Type','severity'
]
# removed tail_number

y = all_flights['arrdel15']

In [4]:
# False positives and false negatives are equally important
# Classes are imbalanced
# Use F1 as metric

## Logistic Regression

### Features: Type

In [5]:
X1 = pd.get_dummies(all_flights['Type']).drop(columns=['None'])

X1

,Fog,Rain,Snow
0,0,0,0
1,0,0,0
2,0,0,0
3,0,1,0
4,0,0,0
...,...,...,...
29730,0,0,0
29731,0,0,0
29732,0,0,0
29733,0,0,0


In [25]:
# 'Type' in X

X_tv, X_test, y_tv, y_test = train_test_split(X1, y, test_size=0.20)

#### Adjust Class Weights

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

lr_type = LogisticRegression(class_weight='balanced').fit(X_train,y_train)

f1_score(y_train, lr_type.predict(X_train)) # 0.215
accuracy_score(y_train, lr_type.predict(X_train)) # 0.790

0.7892494815313043

In [8]:
cross_val_score(lr_type, X_val, y_val, cv=5, scoring='f1')

array([0.24749164, 0.25903614, 0.16718266, 0.21752266, 0.22018349])

In [10]:
confusion_matrix(y_train, lr_type.predict(X_train))

'''
         pred
        |  0  |  1 |
     ___________________
true  0 |  TN | FP |
      1 |  FN | TP |
'''

array([[13552,  1372],
       [ 2456,   461]])

#### Oversampling

In [11]:
y.value_counts()

0    24892
1     4843
Name: arrdel15, dtype: int64

In [5]:
adj = RandomOverSampler(sampling_strategy = {1: np.sum(y_tv==1)*4, 0: np.sum(y_tv==0)})

In [ ]:
X_train_adj, y_train_adj = adj.fit_sample(X_train, y_train)

In [30]:
lr_type = LogisticRegression().fit(X_train_adj,y_train_adj)

f1_score(y_train_adj, lr_type.predict(X_train_adj)) # 0.262
accuracy_score(y_train_adj, lr_type.predict(X_train_adj)) # 0.591

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (15560) in class 1 will be larger than the number of samples in the majority class (class #0 -> 14916)
  n_samples_majority,
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (19898) in class 0 will be larger than the number of samples in the majority class (class #0 -> 14916)
  n_samples_majority,


0.5874837836313385

In [34]:
cross_val_score(lr_type, X_val, y_val, cv=5, scoring='f1')

array([0.04040404, 0.06896552, 0.04060914, 0.06965174, 0.05050505])

In [32]:
confusion_matrix(y_train_adj, lr_type.predict(X_train_adj))

'''
         pred
        |  0  |  1 |
     ___________________
true  0 |  TN | FP |
      1 |  FN | TP |
'''

array([[18098,  1800],
       [12827,  2733]])

### Features: Type, operating_airline

In [33]:
X2 = pd.get_dummies(all_flights[['Type','operating_airline']]).drop(
    columns=['Type_None','operating_airline_AS']
)

X2

,Type_Fog,Type_Rain,Type_Snow,operating_airline_9E,operating_airline_AA,operating_airline_B6,operating_airline_DL,operating_airline_EV,operating_airline_F9,operating_airline_G7,operating_airline_MQ,operating_airline_NK,operating_airline_OH,operating_airline_OO,operating_airline_UA,operating_airline_WN,operating_airline_YV,operating_airline_YX
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29730,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [35]:
X_tv, X_test, y_tv, y_test = train_test_split(X2, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

X_train_adj, y_train_adj = adj.fit_sample(X_train, y_train)

lr_2 = LogisticRegression().fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, lr_2.predict(X_train_adj)) # 0.3585
#accuracy_score(y_train_adj, lr_2.predict(X_train_adj)) # 0.6083

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (15560) in class 1 will be larger than the number of samples in the majority class (class #0 -> 14960)
  n_samples_majority,
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (19898) in class 0 will be larger than the number of samples in the majority class (class #0 -> 14960)
  n_samples_majority,


0.3496923871694923

In [36]:
cross_val_score(lr_2, X_val, y_val, cv=5, scoring='f1')

array([0.04040404, 0.06896552, 0.04      , 0.06896552, 0.07881773])

### Features: Type, marketing_airline_network, operating_airline

In [37]:
X3 = pd.get_dummies(all_flights[['Type','marketing_airline_network','operating_airline']]).drop(
    columns=['Type_None','marketing_airline_network_AS','operating_airline_AS']
)

X3

,Type_Fog,Type_Rain,Type_Snow,marketing_airline_network_AA,marketing_airline_network_B6,marketing_airline_network_DL,marketing_airline_network_F9,marketing_airline_network_NK,marketing_airline_network_UA,marketing_airline_network_WN,...,operating_airline_F9,operating_airline_G7,operating_airline_MQ,operating_airline_NK,operating_airline_OH,operating_airline_OO,operating_airline_UA,operating_airline_WN,operating_airline_YV,operating_airline_YX
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29730,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
29731,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
29732,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
29733,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
X_tv, X_test, y_tv, y_test = train_test_split(X3, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)
X_train_adj, y_train_adj = adj.fit_sample(X_train, y_train)

lr_3 = LogisticRegression().fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, lr_3.predict(X_train_adj)) # 0.3621
accuracy_score(y_train_adj, lr_3.predict(X_train_adj)) # 0.6093

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (15560) in class 1 will be larger than the number of samples in the majority class (class #0 -> 14962)
  n_samples_majority,
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (19898) in class 0 will be larger than the number of samples in the majority class (class #0 -> 14962)
  n_samples_majority,


0.6053922951097073

In [39]:
cross_val_score(lr_3, X_val, y_val, cv=5, scoring='f1')

# better scores w/out marketing airline

array([0.09852217, 0.06030151, 0.08910891, 0.05825243, 0.03015075])

In [43]:
confusion_matrix(y_train_adj, lr_3.predict(X_train_adj))

'''
         pred
        |  0  |  1 |
     ___________________
true  0 |  TN | FP |
      1 |  FN | TP |
'''

array([[16234,  3664],
       [10328,  5232]])

# Model Comparison

In [15]:
X_all = pd.get_dummies(all_flights[features+['dest']], drop_first=True)

X_tv, X_test, y_tv, y_test = train_test_split(X_all, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

X_train_adj, y_train_adj = adj.fit_sample(X_train, y_train)

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (15372) in class 1 will be larger than the number of samples in the majority class (class #0 -> 14919)
  n_samples_majority,
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (19945) in class 0 will be larger than the number of samples in the majority class (class #0 -> 14919)
  n_samples_majority,


### Logistic Regression

In [16]:
lr_all = LogisticRegression().fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, lr_all.predict(X_train_adj))

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.46270795192231157

In [17]:
roc_auc_score(y_train_adj, lr_all.predict_proba(X_train_adj)[:,1])

0.6491520918800445

In [18]:
cross_val_score(lr_all, X_val, y_val, cv=5, scoring='roc_auc')

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

array([0.60927458, 0.65211184, 0.60507952, 0.60827271, 0.62415519])

In [19]:
lr_all_cw = LogisticRegression(class_weight='balanced').fit(X_train, y_train)

f1_score(y_train, lr_all_cw.predict(X_train))

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.334398721534159

In [20]:
roc_auc_score(y_train, lr_all_cw.predict_proba(X_train)[:,1])

0.6538412951269275

In [21]:
cross_val_score(lr_all_cw, X_val, y_val, cv=5, scoring='roc_auc')

/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/wasilaq/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

array([0.59846154, 0.65163252, 0.59864529, 0.60313935, 0.61981633])

### Random Forest

In [22]:
rfc_all = RandomForestClassifier().fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, rfc_all.predict(X_train_adj))

0.8921583315854134

In [23]:
roc_auc_score(y_train_adj, rfc_all.predict_proba(X_train_adj)[:,1])

0.9646226576637666

In [24]:
cross_val_score(rfc_all, X_val, y_val, cv=5, scoring='f1')

array([0.12318841, 0.20979021, 0.16494845, 0.17307692, 0.17266187])

In [25]:
cross_val_score(rfc_all, X_val, y_val, cv=5, scoring='roc_auc')

array([0.63026156, 0.59833527, 0.60178729, 0.62570809, 0.6090956 ])

In [26]:
rfc_all_cw = RandomForestClassifier(class_weight='balanced').fit(X_train, y_train)

In [29]:
f1_score(y_train_adj, rfc_all_cw.predict(X_train_adj))

0.8479127196707389

In [27]:
cross_val_score(rfc_all_cw, X_val, y_val, cv=5, scoring='f1')

array([0.15333333, 0.21935484, 0.20754717, 0.19526627, 0.20327869])

In [28]:
cross_val_score(rfc_all_cw, X_val, y_val, cv=5, scoring='roc_auc')

array([0.62827213, 0.60642958, 0.60216029, 0.61913275, 0.63925347])

### KNN

In [30]:
knn_all = KNeighborsClassifier(n_neighbors=100).fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, knn_all.predict(X_train_adj))

0.4932814846186424

In [31]:
roc_auc_score(y_train_adj, knn_all.predict_proba(X_train_adj)[:,1])

0.6706124153417735

In [32]:
cross_val_score(knn_all, X_val, y_val, cv=5, scoring='roc_auc')

array([0.54957866, 0.57538462, 0.54939647, 0.5234587 , 0.56271217])

### Naive Bayes

In [33]:
nb_all = BernoulliNB().fit(X_train_adj, y_train_adj)

f1_score(y_train_adj, nb_all.predict(X_train_adj)) # 0.6073

0.4490168818272096

In [34]:
roc_auc_score(y_train_adj, nb_all.predict_proba(X_train_adj)[:,1]) # 0.7617

0.6349646947398345

In [35]:
cross_val_score(rfc_all, X_val, y_val, cv=5, scoring='f1')

array([0.1294964 , 0.22145329, 0.16722408, 0.18649518, 0.17518248])

In [36]:
cross_val_score(rfc_all, X_val, y_val, cv=5, scoring='roc_auc')

array([0.61894086, 0.59966757, 0.60211107, 0.60604654, 0.63108652])

# Random Forest

## Number of Trees

In [39]:
rfc_150 = RandomForestClassifier(n_estimators=150, class_weight='balanced').fit(X_train, y_train)

f1_score(y_train, rfc_150.predict(X_train))

0.6997051804015162

In [40]:
cross_val_score(rfc_150, X_val, y_val, cv=5, scoring='f1')

array([0.1650165 , 0.20261438, 0.20754717, 0.2042042 , 0.20261438])

In [41]:
rfc_75 = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train, y_train)

f1_score(y_train, rfc_75.predict(X_train))

0.6976013706453454

In [42]:
cross_val_score(rfc_75, X_val, y_val, cv=5, scoring='f1')

array([0.17880795, 0.2278481 , 0.19745223, 0.23255814, 0.23566879])

In [44]:
rfc_50 = RandomForestClassifier(n_estimators=50, class_weight='balanced').fit(X_train, y_train)

f1_score(y_train, rfc_50.predict(X_train))

0.6921981776765377

In [45]:
cross_val_score(rfc_50, X_val, y_val, cv=5, scoring='f1')

array([0.15384615, 0.2       , 0.17647059, 0.21752266, 0.2020202 ])

In [46]:
rfc_40 = RandomForestClassifier(n_estimators=40, class_weight='balanced').fit(X_train, y_train)

f1_score(y_train, rfc_40.predict(X_train))

0.6898434133026864

In [47]:
cross_val_score(rfc_40, X_val, y_val, cv=5, scoring='f1')

array([0.19269103, 0.24516129, 0.17763158, 0.22089552, 0.21192053])

In [ ]:
# use 75 trees

### Features: dayofweek, operating_airline, Type

In [60]:
X_r1 = pd.get_dummies(all_flights[['dayofweek','operating_airline','Type']], drop_first=True)

X_tv, X_test, y_tv, y_test = train_test_split(X_r1, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

In [ ]:
rfc_1 = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train, y_train)

In [51]:
print(f1_score(y_train, rfc_1.predict(X_train)))
cross_val_score(rfc_1, X_val, y_val, cv=5, scoring='f1')

0.3547838539335899


array([0.31485588, 0.31827957, 0.31355932, 0.28711256, 0.30210325])

In [61]:
cross_val_score(rfc_1, X_val, y_val, cv=5, scoring='accuracy')

array([0.64705882, 0.65210084, 0.64844407, 0.67788057, 0.65937763])

### Add destination

In [115]:
X_r2 = pd.get_dummies(all_flights[['dayofweek','operating_airline','Type','dest']], drop_first=True)

X_tv, X_test, y_tv, y_test = train_test_split(X_r2, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

In [116]:
rfc_2 = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train, y_train)

In [117]:
print(f1_score(y_train, rfc_2.predict(X_train)))
cross_val_score(rfc_2, X_val, y_val, cv=5, scoring='f1')

0.46929722157366327


array([0.2642369 , 0.23414634, 0.24      , 0.26388889, 0.23788546])

In [118]:
cross_val_score(rfc_2, X_val, y_val, cv=5, scoring='accuracy')

array([0.73109244, 0.73697479, 0.72497897, 0.7392767 , 0.71404542])

In [119]:
X_r3 = pd.get_dummies(all_flights[['dayofweek','operating_airline','Type','destcityname']], drop_first=True)

X_tv, X_test, y_tv, y_test = train_test_split(X_r3, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

rfc_3 = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train, y_train)

In [120]:
print(f1_score(y_train, rfc_3.predict(X_train)))
cross_val_score(rfc_3, X_val, y_val, cv=5, scoring='f1')

# destination features resulting in lower validation scores

0.4697612410616894


array([0.248394  , 0.32478632, 0.24842105, 0.24282561, 0.26521739])

### Features: dayofweek, operating_airline, tail_number, Type, deptimeblk, arrtimeblk

In [58]:
X_r4 = pd.get_dummies(all_flights[['dayofweek','operating_airline','Type','arrtimeblk','deptimeblk']], drop_first=True)

X_tv, X_test, y_tv, y_test = train_test_split(X_r4, y, test_size=0.20)
X_train, X_val, y_train, y_test = train_test_split(X_tv, y_tv, test_size=0.25)

rfc_4 = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train, y_train)

In [59]:
print(f1_score(y_train, rfc_4.predict(X_train)))
cross_val_score(rfc_4, X_val, y_val, cv=5, scoring='f1')

# adding arrival and departure times lowered score

0.47025695177754306


array([0.17029703, 0.19341564, 0.17131474, 0.19885277, 0.19639279])

Best feature combo
* dayofweek
* operating_airline
* Type

### Add Oversampling

In [122]:
X_tv, X_test, y_tv, y_test = train_test_split(X_r1, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25)

In [123]:
adj = RandomOverSampler(sampling_strategy = {1: np.sum(y_train==1)*4, 0: np.sum(y_train==0)})
X_train_adj, y_train_adj = adj.fit_sample(X_train, y_train)

In [134]:
rfc_1_cw = RandomForestClassifier(n_estimators=75, class_weight='balanced').fit(X_train_adj, y_train_adj)

In [129]:
f1_score(y_train_adj, rfc_1_cw.predict(X_train_adj))

0.5941534105105356

In [125]:
cross_val_score(rfc_1_cw, X_val, y_val, cv=5, scoring='f1')

array([0.30670927, 0.31759657, 0.34035656, 0.32069971, 0.3354232 ])

In [126]:
cross_val_score(rfc_1_cw, X_val, y_val, cv=5, scoring='accuracy')

array([0.58823529, 0.59747899, 0.66947014, 0.60723297, 0.64339781])

## RF Parameters

### Max depth (pruning)

In [127]:
rfc_5 = RandomForestClassifier(n_estimators=75, max_depth=50, class_weight='balanced').fit(X_train_adj, y_train_adj)

In [128]:
print(f1_score(y_train_adj, rfc_5.predict(X_train_adj)))
cross_val_score(rfc_5, X_val, y_val, cv=5, scoring='f1')

0.5943584623065402


array([0.30625832, 0.31746032, 0.34680135, 0.31441048, 0.33385335])

In [130]:
rfc_6 = RandomForestClassifier(n_estimators=75, max_depth=30, class_weight='balanced').fit(X_train_adj, y_train_adj)

In [131]:
print(f1_score(y_train_adj, rfc_6.predict(X_train_adj)))
cross_val_score(rfc_6, X_val, y_val, cv=5, scoring='f1')

0.5938893751823601


array([0.30898876, 0.3045388 , 0.3360522 , 0.31683168, 0.33385335])

In [132]:
rfc_7 = RandomForestClassifier(n_estimators=75, max_depth=20, class_weight='balanced').fit(X_train_adj, y_train_adj)

In [133]:
print(f1_score(y_train_adj, rfc_7.predict(X_train_adj)))
cross_val_score(rfc_7, X_val, y_val, cv=5, scoring='f1')

0.5938893751823601


array([0.31086657, 0.3137931 , 0.34369885, 0.31954351, 0.33700787])

# Final

#### Random forest, 75 trees, class weight balanced, oversampling (x4), features: dayofweek, operating_airline, type

### Training Data

In [135]:
f1_score(y_train, rfc_1_cw.predict(X_train))

0.3579054397559736

### Validation

In [136]:
cross_val_score(rfc_1_cw, X_val, y_val, scoring='f1')

array([0.31024931, 0.31366906, 0.33333333, 0.31625183, 0.33513514])

In [137]:
confusion_matrix(y_val, rfc_1_cw.predict(X_val))

array([[3199, 1827],
       [ 390,  531]])

### Test Data

In [138]:
test_pred = rfc_1_cw.predict(X_test)

In [139]:
f1_score(y_test,test_pred)

0.3283847980997625

In [140]:
confusion_matrix(y_test, test_pred)

array([[3132, 1825],
       [ 437,  553]])

# Exporting Results

In [142]:
pred_prob = rfc_1_cw.predict_proba(X_test)[:,1]

In [143]:
results = all_flights[['dayofweek','operating_airline','Type','arrdel15']].iloc[X_test.index]
results['prediction'] = test_pred
results['pred_prob_delay'] = pred_prob

In [144]:
results

,dayofweek,operating_airline,Type,arrdel15,prediction,pred_prob_delay
10364,1,AA,None,0,0,0.462887
27815,2,DL,None,0,0,0.375174
12965,4,DL,None,0,1,0.582925
13541,5,DL,None,0,0,0.425104
16038,5,DL,None,0,0,0.425104
...,...,...,...,...,...,...
22133,1,DL,None,0,1,0.505356
17870,5,DL,Rain,1,1,0.864576
17207,4,DL,Rain,0,0,0.273748
6682,1,WN,None,0,1,0.546635


In [149]:
results.to_csv('/Users/wasilaq/Metis/Proj3/pickled_files/ArrivalDelayResults.csv')

In [4]:
df = pd.read_csv('/Users/wasilaq/Metis/Proj3/pickled_files/ArrivalDelayResults.csv')

df.head()

,Unnamed: 0,dayofweek,operating_airline,Type,arrdel15,prediction,pred_prob_delay
0,10364,1,AA,None,0,0,0.462887
1,27815,2,DL,None,0,0,0.375174
2,12965,4,DL,None,0,1,0.582925
3,13541,5,DL,None,0,0,0.425104
4,16038,5,DL,None,0,0,0.425104


# Saving Info For Tableau

In [9]:
add_info = all_flights[
    ['dep','flightdate','dayofmonth_x','dest','destcityname','StartTime(UTC)','EndTime(UTC)']
].iloc[df['Unnamed: 0']]

add_info.head()

add_info.to_csv('/Users/wasilaq/Metis/Proj3/pickled_files/add_info.csv')